# Master Table CCLE N28

Este notebook contiene los procedimientos realizados para lograr obtener una master table con todas las tablas de reditools correspondientes a las 28 lineas celulares de CCLE.

A continuación se muestra el contenido de la carpeta `../data/`, donde se encuentrar los archivos de reditools (`*.tsv`) junto a sus archivos log de reditools (`*_log.txt`):

```bash
tree ../data/
.
├── C836.HCC1143.2_log.txt
├── C836.HCC1143.2.tsv
├── C836.HCC1395.2_log.txt
├── C836.HCC1395.2.tsv
├── C836.JIMT-1.1_log.txt
├── C836.JIMT-1.1.tsv
├── C836.MDA-MB-415.1_log.txt
├── C836.MDA-MB-415.1.tsv
├── G20496.HCC1954.2_log.txt
├── G20496.HCC1954.2.tsv
├── G26176.CAL-51.2_log.txt
├── G26176.CAL-51.2.tsv
├── G26185.HCC70.2_log.txt
├── G26185.HCC70.2.tsv
├── G26206.CAL-120.2_log.txt
├── G26206.CAL-120.2.tsv
├── G26250.BT-474.2_log.txt
├── G26250.BT-474.2.tsv
├── G26260.HCC1395.2_log.txt
├── G26260.HCC1395.2.tsv
├── G27249.AU565.1_log.txt
├── G27249.AU565.1.tsv
├── G27278.ZR-75-30.1_log.txt
├── G27278.ZR-75-30.1.tsv
├── G27300.BT-483.1_log.txt
├── G27300.BT-483.1.tsv
├── G27310.HCC1428.1_log.txt
├── G27310.HCC1428.1.tsv
├── G27322.HCC1500.1_log.txt
├── G27322.HCC1500.1.tsv
├── G27334.CAMA-1.1_log.txt
├── G27334.CAMA-1.1.tsv
├── G27355.EFM-192A.1_log.txt
├── G27355.EFM-192A.1.tsv
├── G27365.CAL-148.1_log.txt
├── G27365.CAL-148.1.tsv
├── G28036.JIMT-1.1_log.txt
├── G28036.JIMT-1.1.tsv
├── G28064.MDA-MB-468.1_log.txt
├── G28064.MDA-MB-468.1.tsv
├── G28069.MDA-MB-415.1_log.txt
├── G28069.MDA-MB-415.1.tsv
├── G28072.MDA-MB-175-VII.1_log.txt
├── G28072.MDA-MB-175-VII.1.tsv
├── G28869.HCC38.3_log.txt
├── G28869.HCC38.3.tsv
├── G28898.HCC202.3_log.txt
├── G28898.HCC202.3.tsv
├── G30590.BT-549.1_log.txt
├── G30590.BT-549.1.tsv
├── G30624.UACC-812.1_log.txt
├── G30624.UACC-812.1.tsv
├── G41720.HCC2218.5_log.txt
├── G41720.HCC2218.5.tsv
├── G41743.HCC1419.5_log.txt
└── G41743.HCC1419.5.tsv

```


# PROCEDIMIENTOS

## Filtering of samples

Filters applied (in order):
1. (coverage at position >= 10)
2. (var == "AG" OR var == "TC")
3. (coverage of alt. allele >= 5)
4. (freq. of alt. allele >= 0.05)               

In [17]:
# Import the libraries
import pandas as pd
import glob
import sys
import numpy as np
import os

# Function to filter df by coverage and alleles
def filtering(path_file, out_dir):
    
    # Create the out_file name
    out_file = os.path.join(out_dir, os.path.basename(path_file).replace('.tsv', '_filtered.tsv'))
    
    count = 0
    with open(path_file, 'r') as fp , open(out_file, 'w+') as fp_out:
        for line in fp:
            if count == 0:
                fp_out.write(f"{line}")
                count += 1
                continue
            count += 1
            tmp = line.split('\t')
            if (int(tmp[4]) >= 10) & (('AG' in tmp[7]) | ('TC' in tmp[7])):
                if ((tmp[2] == 'A') & (int(tmp[6].split(',')[2]) >= 5) & ( int(tmp[6].split(',')[2])/int(tmp[4]) >= 0.05 ) ) | ((tmp[2] == 'T') & (int(tmp[6].split(',')[1]) >= 5) & ( int(tmp[6].split(',')[1])/int(tmp[4]) >= 0.05 )):
                    fp_out.write(f"{line}")
            #if count == 5:
            #    break
    
    print('------------------------------------')


# Get list of reditools tables
tables = glob.glob(os.path.join('../data', '*.tsv'))

# Create the out_dir and out_file
out_dir = '../outputs/01_filtering_tables'
os.makedirs(out_dir, exist_ok=True)

for file in tables:
    print('Filtering file: {}'.format(os.path.basename(file)))
    filtering(file, out_dir)

#filtering(tables[0], out_dir)


Filtering file: G28036.JIMT-1.1.tsv
------------------------------------
Filtering file: G26185.HCC70.2.tsv
------------------------------------
Filtering file: C836.HCC1143.2.tsv
------------------------------------
Filtering file: G26176.CAL-51.2.tsv
------------------------------------
Filtering file: G28898.HCC202.3.tsv
------------------------------------
Filtering file: C836.MDA-MB-415.1.tsv
------------------------------------
Filtering file: C836.HCC1395.2.tsv
------------------------------------
Filtering file: G27310.HCC1428.1.tsv
------------------------------------
Filtering file: G30590.BT-549.1.tsv
------------------------------------
Filtering file: C836.JIMT-1.1.tsv
------------------------------------
Filtering file: G41720.HCC2218.5.tsv
------------------------------------
Filtering file: G28064.MDA-MB-468.1.tsv
------------------------------------
Filtering file: G26260.HCC1395.2.tsv
------------------------------------
Filtering file: G28072.MDA-MB-175-VII.1.tsv
---

Hay 4 muestras que se cayeron durante el análisis. Asi que serán eliminadas de la carpeta `01_filtering_tables`. Éstas 4 muestras son las siguientes

In [18]:
%%bash

# Get Samples with errors: 
echo "Las siguientes muestras deben ser eliminadas:"

find ../outputs -name "*.tsv" | grep -f <(grep -i "err" ../data/*_log.txt | cut -f1 -d':' | cut -f3 -d'/' | sed 's/_log.txt//g')

# Delete the files
rm $(find ../outputs -name "*.tsv" | grep -f <(grep -i "err" ../data/*_log.txt | cut -f1 -d':' | cut -f3 -d'/' | sed 's/_log.txt//g') | xargs)


Las siguientes muestras deben ser eliminadas:
../outputs/01_filtering_tables/C836.JIMT-1.1_filtered.tsv
../outputs/01_filtering_tables/C836.MDA-MB-415.1_filtered.tsv
../outputs/01_filtering_tables/C836.HCC1395.2_filtered.tsv
../outputs/01_filtering_tables/C836.HCC1143.2_filtered.tsv


El resumen de las variantes filtradas se muestra a continuación:

| cell_line               | raw sites | filtered sites |
|-------------------------|-----------|----------------|
| C836.HCC1143.2          | 223       | NA             |
| C836.HCC1395.2          | 505606    | NA             |
| C836.JIMT-1.1           | 265409    | NA             |
| C836.MDA-MB-415.1       | 608808    | NA             |
| G20496.HCC1954.2        | 4180142   | 16442          |
| G26176.CAL-51.2         | 4443743   | 24098          |
| G26185.HCC70.2          | 4132223   | 20643          |
| G26206.CAL-120.2        | 3361752   | 13527          |
| G26250.BT-474.2         | 3814955   | 17178          |
| G26260.HCC1395.2        | 4112263   | 17521          |
| G27249.AU565.1          | 4909995   | 24463          |
| G27278.ZR-75-30.1       | 4313095   | 27025          |
| G27300.BT-483.1         | 3776925   | 18278          |
| G27310.HCC1428.1        | 4095853   | 20480          |
| G27322.HCC1500.1        | 4179877   | 29917          |
| G27334.CAMA-1.1         | 3850742   | 22348          |
| G27355.EFM-192A.1       | 4011857   | 17354          |
| G27365.CAL-148.1        | 4554028   | 16963          |
| G28036.JIMT-1.1         | 3823738   | 16238          |
| G28064.MDA-MB-468.1     | 3499509   | 18532          |
| G28069.MDA-MB-415.1     | 4358917   | 26397          |
| G28072.MDA-MB-175-VII.1 | 4582056   | 29807          |
| G28869.HCC38.3          | 5794550   | 27958          |
| G28898.HCC202.3         | 4968291   | 25400          |
| G30590.BT-549.1         | 3087657   | 13112          |
| G30624.UACC-812.1       | 880975    | 5732           |
| G41720.HCC2218.5        | 5187105   | 23774          |
| G41743.HCC1419.5        | 5567992   | 25417          |

## Multiple Merging
The 24 final tables were merged into One Big Master Table (code below)


In [23]:

# Import the libraries 
import pandas as pd
import os
import sys
import numpy as np
import glob
from functools import reduce


# Table's path
tables_path = '../outputs/01_filtering_tables'

# Get the table list
tables = glob.glob(os.path.join(tables_path, '*.tsv'))
# Get a list with columns names
cols = 'Region','Position','Reference','Strand','Coverage-q30','MeanQ','BaseCount[A,C,G,T]','AllSubs','Frequency','gCoverage-q30','gMeanQ','gBaseCount[A,C,G,T]','gAllSubs','gFrequency'

# Create a list with dataframes
tables_df = []
for table in tables:
    # Open the table
    df = pd.read_csv(table, sep="\t", header=0)
    # add_sample_names as column-suffix
    id = os.path.basename(table).split('_')[0]
    cols_tmp = [col+'_'+id if col not in ['Region', 'Position', 'Reference'] else col for col in cols ]
    df.columns = cols_tmp
    # Sort the df
    df.sort_values(by = ['Region', 'Position'], ascending = [True, True], na_position = 'last', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    # Change some column types
    df = df.astype({'Region':'string','Position':'int64', 'Reference':'string'})
    # Append df to list
    tables_df.append(df)

# Merge the tables
tables_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Region', 'Position', 'Reference'], how='outer'), tables_df).fillna('NA')
 
tables_merged.head()



,Region,Position,Reference,Strand_G28069.MDA-MB-415.1,Coverage-q30_G28069.MDA-MB-415.1,MeanQ_G28069.MDA-MB-415.1,"BaseCount[A,C,G,T]_G28069.MDA-MB-415.1",AllSubs_G28069.MDA-MB-415.1,Frequency_G28069.MDA-MB-415.1,gCoverage-q30_G28069.MDA-MB-415.1,...,Coverage-q30_G20496.HCC1954.2,MeanQ_G20496.HCC1954.2,"BaseCount[A,C,G,T]_G20496.HCC1954.2",AllSubs_G20496.HCC1954.2,Frequency_G20496.HCC1954.2,gCoverage-q30_G20496.HCC1954.2,gMeanQ_G20496.HCC1954.2,"gBaseCount[A,C,G,T]_G20496.HCC1954.2",gAllSubs_G20496.HCC1954.2,gFrequency_G20496.HCC1954.2
0,1,14907,A,2.0,39.0,36.95,"[9, 0, 30, 0]",AG,0.77,-,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,1,14930,A,2.0,47.0,37.6,"[9, 0, 38, 0]",AG,0.81,-,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1,15118,A,2.0,11.0,35.09,"[5, 0, 6, 0]",AG,0.55,-,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1,15447,A,2.0,29.0,36.93,"[17, 0, 12, 0]",AG,0.41,-,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,16378,T,2.0,13.0,38.15,"[0, 12, 0, 1]",TC,0.92,-,...,11.0,38.36,"[0, 11, 0, 0]",TC,1.0,-,-,-,-,-


In [25]:
# Create outdir
os.makedirs('../outputs/02_master_table/', exist_ok=True)
# Save the table to Excel
tables_merged.to_excel('../outputs/02_master_table/CCLE24_master_table.xlsx', header=True)
# Save table to tsv
tables_merged.to_csv('../outputs/02_master_table/CCLE24_master_table.tsv', sep='\t', header=True, index=False)

In [26]:
# Create minimalist table
# Get a list with columns names
cols = tables_merged.columns
# Keep only the minimal columns
cols2keep = ['Region', 'Position', 'Reference']+[col for col in cols if (('Coverage-q30' in col) | ('BaseCount[A,C,G,T]' in col) | ('Strand' in col)) ]
# Remove the "g" columns
cols2keep = [col for col in cols2keep if (('gBaseCount[A,C,G,T]' not in col) & ('gCoverage-q30' not in col)) ]

# Create the minimal table and save it to file
tables_merged_minimal = tables_merged[cols2keep].copy()

# Save to file
tables_merged_minimal.to_excel('../outputs/02_master_table/CCLE24_master_table_minimal.xlsx', header=True)
# Save table to tsv
tables_merged_minimal.to_csv('../outputs/02_master_table/CCLE24_master_table_minimal.tsv', sep='\t', header=True, index=False)

# Preview table
tables_merged_minimal

,Region,Position,Reference,Strand_G28069.MDA-MB-415.1,Coverage-q30_G28069.MDA-MB-415.1,"BaseCount[A,C,G,T]_G28069.MDA-MB-415.1",Strand_G30624.UACC-812.1,Coverage-q30_G30624.UACC-812.1,"BaseCount[A,C,G,T]_G30624.UACC-812.1",Strand_G26260.HCC1395.2,...,"BaseCount[A,C,G,T]_G28869.HCC38.3",Strand_G30590.BT-549.1,Coverage-q30_G30590.BT-549.1,"BaseCount[A,C,G,T]_G30590.BT-549.1",Strand_G28036.JIMT-1.1,Coverage-q30_G28036.JIMT-1.1,"BaseCount[A,C,G,T]_G28036.JIMT-1.1",Strand_G20496.HCC1954.2,Coverage-q30_G20496.HCC1954.2,"BaseCount[A,C,G,T]_G20496.HCC1954.2"
0,1,14907,A,2.0,39.0,"[9, 0, 30, 0]",NA,NA,NA,NA,...,NA,2.0,13.0,"[7, 0, 6, 0]",NA,NA,NA,NA,NA,NA
1,1,14930,A,2.0,47.0,"[9, 0, 38, 0]",NA,NA,NA,NA,...,"[20, 0, 9, 0]",2.0,11.0,"[5, 0, 6, 0]",NA,NA,NA,NA,NA,NA
2,1,15118,A,2.0,11.0,"[5, 0, 6, 0]",NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1,15447,A,2.0,29.0,"[17, 0, 12, 0]",NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,16378,T,2.0,13.0,"[0, 12, 0, 1]",2.0,17.0,"[0, 7, 0, 10]",2.0,...,"[0, 11, 0, 1]",2.0,19.0,"[0, 12, 0, 7]",NA,NA,NA,2.0,11.0,"[0, 11, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136242,X,133561561,T,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2.0,102.0,"[0, 6, 0, 96]"
136243,X,135046009,A,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2.0,133.0,"[1, 0, 132, 0]"
136244,X,148678982,A,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2.0,23.0,"[0, 0, 23, 0]"
136245,X,149997252,A,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,2.0,13.0,"[0, 0, 13, 0]"


# Counting reads with bam-readcount

GitHub of the project: [bam-readcount](https://github.com/genome/bam-readcount)

Tutorial: [bam-readcount Tutorial](https://github.com/genome/bam-readcount/tree/master/tutorial)

Installation of the software was donde following github instructions for compilig method on YB's VM.

## Create the regions list

In [ ]:
%%bash

# table with editions
editions="../outputs/02_master_table/CCLE24_master_table_minimal.tsv"
awk 'NR>1{print $1"\t"$2"\t"$2}' $editions > ../outputs/02_master_table/CCLE24_master_table_minimal.regions

## Run bam-readcount
`bam-readcount` fue ejecutado para cada linea celular, utilizando la lista de posiciones creada en el paso anterior.

El comando utilizado se muestra a continuación:

```bash
    bam-readcount -b 30 -q 20 -w1 -f ${reference_fasta} -l ${regions_list} ${bam_file} > ${outdir}/${sample}.tsv
```

Donde:

  * -b 30: Minimun base quality of 30.
  * -q 20: Minumun mapping quality of 20.
  * -w1: to show warnings just once.
  * -f: Genoma de referencia. En este caso se utilizó el archivo del repositorio del Broad Institute (Link al archivo)[https://data.broadinstitute.org/snowman/hg19/Homo_sapiens_assembly19.fasta].
  * -l: Es la lista de posiciones de interés.

Esto se ejecutó mediante los scripts `bam-readcount_parallel.sh` y `bam-readcount.sh`.

```bash

cd ../code
./bam-readcount_parallel.sh

```

Algunas aclaraciones:

- El comando anterior ejecutará el script analizando 8 muestras a la vez. Si se desea cambiar este valor, se debe modificar la variable `MAX_NPROC=8`.
- La carpeta de input y output están asignadas directamente en las lineas:  
     50: `find ../../CCLE_N28/bams -name "*.bam" | while read file;`  
     44: `out_dir="../outputs/04_bam-readcount"` 


## bam-readcount Results Post-processing.

Se dejará solamente la cobertura total del sitio y de cada alelo, junto a su respectiva cobertura.   
De acuerdo a la documentación, las columnas de output representan lo siguiente:

> Output
> ------
> 
> Output is tab-separated with no header to `STDOUT`, one line per
> position:
> 
>     chr	position	reference_base	depth	base:count:avg_mapping_quality:avg_basequality:avg_se_mapping_quality:num_plus_strand:num_minus_strand:avg_pos_as_fraction:avg_num_mismatches_as_fraction:avg_sum_mismatch_qualities:num_q2_containing_reads:avg_distance_to_q2_start_in_q2_reads:avg_clipped_length:avg_distance_to_effective_3p_end   ...
> 
> There is one set of `:`-separated fields for each reported `base` with
> statistics on the set of reads containing that base:
> 
> Field | Description
> ----- | -----------
> base | The base, eg `C`
> count | Number of reads
> avg_mapping_quality | Mean mapping quality
> avg_basequality | Mean base quality
> avg_se_mapping_quality | Mean single ended mapping quality
> num_plus_strand | Number of reads on the plus/forward strand
> num_minus_strand | Number of reads on the minus/reverse strand
> avg_pos_as_fraction | Average position on the read as a fraction, calculated with respect to the length after clipping. This value is normalized to the center of the read: bases occurring strictly at the center of the read have a value of 1, those occurring strictly at the ends should approach a value of 0
> avg_num_mismatches_as_fraction | Average number of mismatches on these reads per base
> avg_sum_mismatch_qualities | Average sum of the base qualities of mismatches in the reads
> num_q2_containing_reads | Number of reads with q2 runs at the 3’ end
> avg_distance_to_q2_start_in_q2_reads | Average distance of position (as fraction of unclipped read length) to the start of the q2 run
> avg_clipped_length | Average clipped read length
> avg_distance_to_effective_3p_end | Average distance to the 3’ prime end of the read (as fraction of unclipped read length)



Sólo nos quedaremos con:  
        `chr position	reference_base	depth   base:count`

In [1]:
# Import the libraries
import pandas as pd
import glob
import sys
import numpy as np
import os

# Function to get coverage per site and per nucleotide from bam-readcount output table
def reduce_table(path_table, out_dir):
    
    # Create the out_file name
    out_file = os.path.join(out_dir, os.path.basename(path_table).replace('.tsv', '_minimal.tsv'))
    
    count = 0
    with open(path_table, 'r') as fp , open(out_file, 'w+') as fp_out:
        for line in fp:
            count += 1
            tmp_line = line.split('\t')
            tmpA = tmp_line[5].split(':')[1]    # A counts
            tmpC = tmp_line[6].split(':')[1]    # C counts
            tmpG = tmp_line[7].split(':')[1]    # G counts
            tmpT = tmp_line[8].split(':')[1]    # T counts
            # Generate out_line with the base count in the same format as reditools ([A,C,G,T])
            out_line = '{}\t{}\t{}\t{}\t[{},{},{},{}]\n'.format(tmp_line[0], tmp_line[1], tmp_line[2], tmp_line[3], tmpA, tmpC, tmpG, tmpT)
            # write to output file
            fp_out.write(f"{out_line}")
            
    print('------------------------------------')


# Get list of bam-readcount tables
tables = glob.glob(os.path.join('../outputs/04_bam-readcount', '*.tsv'))

# set out_dir
out_dir = '../outputs/04_bam-readcount'

for file in tables:
    print('Reducing file: {}'.format(os.path.basename(file)))
    reduce_table(file, out_dir)



Filtering file: G28036.JIMT-1.1.tsv
------------------------------------
Filtering file: G26185.HCC70.2.tsv
------------------------------------
Filtering file: C836.HCC1143.2.tsv
------------------------------------
Filtering file: G26176.CAL-51.2.tsv
------------------------------------
Filtering file: G28898.HCC202.3.tsv
------------------------------------
Filtering file: C836.MDA-MB-415.1.tsv
------------------------------------
Filtering file: C836.HCC1395.2.tsv
------------------------------------
Filtering file: G27310.HCC1428.1.tsv
------------------------------------
Filtering file: G30590.BT-549.1.tsv
------------------------------------
Filtering file: C836.JIMT-1.1.tsv
------------------------------------
Filtering file: G41720.HCC2218.5.tsv
------------------------------------
Filtering file: G28064.MDA-MB-468.1.tsv
------------------------------------
Filtering file: G26260.HCC1395.2.tsv
------------------------------------
Filtering file: G28072.MDA-MB-175-VII.1.tsv
---

# Merge Master-Table w/ bam-readcount tables
El último paso es unir la master table `CCLE24_master_table_minimal.tsv`, con la información obtenida del software `bam-readcount`.

La Tabla a continuación muestra el número de ediciones que fueron recuperadas en cada línea celular.

Código usado para obtener los conteos:

```bash
    wc -l ../outputs/04_bam-readcount/*minimal.tsv  | sed 's/..\/outputs\/04_bam-readcount\///g' | awk '{print $2"\t"$1}' 

```

| Cell line | Number of editions |
|---|---|
| C836.HCC1143.2_minimal.tsv | 36320 |
| C836.HCC1395.2_minimal.tsv | 37883 |
| C836.JIMT-1.1_minimal.tsv | 36030 |
| C836.MDA-MB-415.1_minimal.tsv | 36606 |
| G20496.HCC1954.2_minimal.tsv | 124523 |
| G26176.CAL-51.2_minimal.tsv | 122034 |
| G26185.HCC70.2_minimal.tsv | 121980 |
| G26206.CAL-120.2_minimal.tsv | 120415 |
| G26250.BT-474.2_minimal.tsv | 124505 |
| G26260.HCC1395.2_minimal.tsv | 122344 |
| G27249.AU565.1_minimal.tsv | 123222 |
| G27278.ZR-75-30.1_minimal.tsv | 123092 |
| G27300.BT-483.1_minimal.tsv | 126672 |
| G27310.HCC1428.1_minimal.tsv | 126807 |
| G27322.HCC1500.1_minimal.tsv | 126778 |
| G27334.CAMA-1.1_minimal.tsv | 124416 |
| G27355.EFM-192A.1_minimal.tsv | 126402 |
| G27365.CAL-148.1_minimal.tsv | 123054 |
| G28036.JIMT-1.1_minimal.tsv | 122959 |
| G28064.MDA-MB-468.1_minimal.tsv | 124777 |
| G28069.MDA-MB-415.1_minimal.tsv | 125989 |
| G28072.MDA-MB-175-VII.1_minimal.tsv | 126914 |
| G28869.HCC38.3_minimal.tsv | 127695 |
| G28898.HCC202.3_minimal.tsv | 124223 |
| G30590.BT-549.1_minimal.tsv | 117085 |
| G30624.UACC-812.1_minimal.tsv | 114631 |
| G41720.HCC2218.5_minimal.tsv | 123190 |
| G41743.HCC1419.5_minimal.tsv | 124882 |

In [27]:
# Import libraries
import pandas as pd
import sys
import os
import glob
import numpy as np
from functools import reduce   # Import reduce function

# Master table CCLE24
mt = '../outputs/02_master_table/CCLE24_master_table_minimal.tsv'
# Read the MT
mt_df = pd.read_csv(mt, sep="\t", header=0)

# bam-readcount tables
tables = glob.glob(os.path.join('../outputs/04_bam-readcount', '*minimal.tsv'))
# remove the C836 files (corrupt files)
tables = [x for x in tables if 'C836' not in x]

# Create a copy of MT
mt_df2 = mt_df.copy()
# Sort the df
mt_df2.sort_values(by = ['Region', 'Position'], ascending = [True, True], na_position = 'last', inplace=True)
# Reset index
mt_df2.reset_index(drop=True, inplace=True)
# Change column types
mt_df2 = mt_df2.astype({'Region':'string','Position':'int64', 'Reference':'string'})

# Merge each bam-readcount table with the MT
for table in tables:
    # Open the table. Columns will be named as: Region  Position    Reference   bam-rc_Coverage-q20    bam-rc[A,C,G,T]
    df = pd.read_csv(table, sep="\t", header=None, names=['Region','Position','Reference','bam-rc_Coverage-q20','bam-rc[A,C,G,T]'])
    # add_sample_names as column-suffix
    id = os.path.basename(table).split('_')[0]
    cols_tmp = [ col+'_'+id if col not in ['Region', 'Position', 'Reference'] else col for col in list(df.columns) ]
    df.columns = cols_tmp
    # Sort the df
    df.sort_values(by = ['Region', 'Position'], ascending = [True, True], na_position = 'last', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    # Change some column types
    df = df.astype({'Region':'string','Position':'int64', 'Reference':'string'})
    # perform the merge
    mt_df2 = mt_df2.merge(df, on=['Region','Position','Reference'], how='left')

# Reorder columns
cols = list(mt_df2.columns)
cols_samples = [col for col in cols if col not in ['Region', 'Position', 'Reference'] ]

# Get uniq ids
ids = [x.split('_')[-1] for x in cols_samples]
ids = list(set(ids))

# Reorder sample cols
cols_new_order = ['Region', 'Position', 'Reference']
for id in ids:
    tmp_cols = [x for x in cols_samples if id in x]
    cols_new_order = cols_new_order + tmp_cols

# final MT
mt_df_final = mt_df2[cols_new_order]

# Save the final table
outdir='../outputs/05_mt-bamreadcount_merge/'
os.makedirs(outdir, exist_ok=True)
mt_df_final.to_csv(os.path.join(outdir, 'CCL24_master_table_minimal_with_bamreadcount.tsv'), sep="\t", header=True, index=False, na_rep='NA')


/tmp/ipykernel_5281/1132225235.py:12: DtypeWarning: Columns (0,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62) have mixed types. Specify dtype option on import or set low_memory=False.
  mt_df = pd.read_csv(mt, sep="\t", header=0)


In [41]:
# Output a minimal version, just with {Region}, {Position}, {Reference}, {BaseCount[A,C,G,T]_*} and {bam-rc[A,C,G,T]_*}
minim_cols = [x for x in mt_df_final.columns if not x.startswith('Strand') and not x.startswith('Coverage') and not x.startswith('bam-rc_Coverage')]
# Keep the minimun columns
mt_df_final_minimal = mt_df_final[minim_cols]
# Save to table
mt_df_final_minimal.to_csv(os.path.join(outdir, 'CCL24_master_table_just-counts_with_bamreadcount.tsv'), sep="\t", header=True, index=False, na_rep='NA')

In [42]:
mt_df_final_minimal

,Region,Position,Reference,"BaseCount[A,C,G,T]_G27334.CAMA-1.1","bam-rc[A,C,G,T]_G27334.CAMA-1.1","BaseCount[A,C,G,T]_G26206.CAL-120.2","bam-rc[A,C,G,T]_G26206.CAL-120.2","BaseCount[A,C,G,T]_G27310.HCC1428.1","bam-rc[A,C,G,T]_G27310.HCC1428.1","BaseCount[A,C,G,T]_G27249.AU565.1",...,"BaseCount[A,C,G,T]_G26260.HCC1395.2","bam-rc[A,C,G,T]_G26260.HCC1395.2","BaseCount[A,C,G,T]_G28072.MDA-MB-175-VII.1","bam-rc[A,C,G,T]_G28072.MDA-MB-175-VII.1","BaseCount[A,C,G,T]_G30624.UACC-812.1","bam-rc[A,C,G,T]_G30624.UACC-812.1","BaseCount[A,C,G,T]_G20496.HCC1954.2","bam-rc[A,C,G,T]_G20496.HCC1954.2","BaseCount[A,C,G,T]_G28069.MDA-MB-415.1","bam-rc[A,C,G,T]_G28069.MDA-MB-415.1"
0,1,14907,A,NaN,"[6,0,3,0]",NaN,"[9,0,3,0]","[4, 0, 12, 0]","[4,0,12,0]","[6, 0, 13, 0]",...,NaN,"[5,0,1,0]","[10, 0, 10, 0]","[10,0,10,0]",NaN,"[4,0,2,0]",NaN,"[0,0,5,0]","[9, 0, 30, 0]","[9,0,30,0]"
1,1,14930,A,NaN,"[8,0,3,0]",NaN,"[9,0,2,0]","[4, 0, 15, 0]","[4,0,15,0]","[6, 0, 11, 0]",...,NaN,"[5,0,1,0]","[8, 0, 19, 0]","[8,0,19,0]",NaN,"[4,0,1,0]",NaN,"[1,0,6,0]","[9, 0, 38, 0]","[9,0,38,0]"
2,1,15118,A,NaN,"[0,0,0,0]",NaN,"[1,0,0,0]",NaN,"[1,0,0,0]",NaN,...,NaN,"[0,0,0,0]",NaN,"[1,0,1,0]",NaN,"[0,0,0,0]",NaN,"[2,0,0,0]","[5, 0, 6, 0]","[5,0,6,0]"
3,1,15447,A,NaN,"[6,0,0,0]",NaN,"[10,0,0,0]",NaN,"[7,0,0,0]",NaN,...,NaN,"[9,0,1,0]",NaN,"[19,0,2,0]",NaN,"[4,0,0,0]",NaN,"[12,0,0,0]","[17, 0, 12, 0]","[17,0,12,0]"
4,1,16378,T,"[0, 15, 0, 13]","[0,15,0,14]",NaN,"[0,3,0,12]","[0, 8, 0, 8]","[0,8,0,8]","[0, 28, 0, 15]",...,"[0, 12, 0, 6]","[0,13,0,6]","[0, 27, 0, 19]","[0,27,0,19]","[0, 7, 0, 10]","[0,7,0,10]","[0, 11, 0, 0]","[0,11,0,0]","[0, 12, 0, 1]","[0,12,0,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136242,Y,21153474,A,"[2, 0, 1033, 0]","[2,0,1048,0]","[0, 0, 110, 0]","[0,0,117,0]","[1, 0, 713, 0]","[1,0,724,0]","[3, 3, 2119, 1]",...,"[0, 0, 24, 0]","[0,0,30,0]","[1, 1, 1304, 0]","[1,1,1306,0]","[0, 0, 370, 0]","[0,0,373,0]","[1, 0, 2900, 1]","[1,0,2903,1]","[5, 2, 2367, 1]","[5,2,2369,1]"
136243,Y,21153609,T,"[2, 2077, 0, 1]","[2,2088,0,1]","[0, 257, 0, 0]","[0,266,0,0]","[0, 1405, 0, 1]","[0,1415,0,1]","[0, 3837, 0, 1]",...,"[0, 90, 0, 0]","[0,94,0,0]","[0, 2296, 0, 0]","[0,2297,0,0]","[0, 445, 0, 0]","[0,451,0,0]","[1, 4965, 0, 2]","[1,4970,0,2]","[0, 4015, 0, 0]","[0,4019,0,0]"
136244,Y,21154569,A,"[1, 0, 2462, 0]","[1,0,2465,0]","[0, 0, 453, 0]","[0,0,458,0]","[0, 0, 1848, 0]","[0,0,1850,0]","[3, 1, 6142, 1]",...,"[0, 0, 165, 0]","[0,0,169,0]","[1, 0, 3501, 1]","[1,0,3502,1]","[0, 0, 565, 0]","[0,0,566,0]","[0, 0, 5800, 4]","[0,0,5802,4]","[3, 0, 4944, 0]","[3,0,4944,0]"
136245,Y,21154706,T,NaN,"[0,0,0,1]",NaN,NaN,NaN,"[0,0,0,0]",NaN,...,NaN,"[0,1,0,0]",NaN,"[0,0,0,7]",NaN,NaN,NaN,"[0,1,0,4]",NaN,"[0,0,0,13]"


La tabla anterior contiene las siguientes columnas:
  * ***Region:*** Chromosome id.
  * ***Position:*** Genomic Position.
  * ***Reference:*** Base on reference genome.
  * ***Basecount[A,C,G,T]:*** Base count from Reditools2 analysis.
  * ***bam-rc[A,C,G,T]:*** Base count from bam-readcount analysis.

# Dividir la tabla entre los 3 grupos de drogas

YB, además requiere generar sub-sets que agrupan las líneas celulares según:
  * La droga del tratamiento.
  * Según si es respondedor o no respondedor a la droga. 

La tabla proporcionada se muestra a continuación:

| Cell_line | PARPi | Anthracyclines | Alkylatingagents |
|---:|---:|---:|---:|
| AU565 | HS | HS | HS |
| BT-474 | LS | LS | LS |
| BT-483 | LS | LS | LS |
| BT-549 | LS | HS | LS |
| CAL-120 | LS | NaN | LS |
| CAL-148 | HS | HS | HS |
| CAL-51 | HS | HS | HS |
| CAMA-1 | HS | HS | NaN |
| EFM-192A | NaN | LS | LS |
| HCC1419 | LS | LS | HS |
| HCC1428 | LS | LS | LS |
| HCC1500 | LS | NaN | HS |
| HCC1937 | LS | LS | LS |
| HCC1954 | HS | LS | LS |
| HCC202 | NaN | LS | NaN |
| HCC2218 | NaN | NaN | HS |
| HCC38 | LS | NaN | LS |
| HCC70 | HS | NaN | HS |
| JIMT-1 | NaN | HS | NaN |
| MCF7 | LS | HS | NaN |
| MDA-MB-175-VII | LS | NaN | LS |
| MDA-MB-231 | HS | HS | HS |
| MDA-MB-436 | LS | HS | LS |
| MDA-MB-468 | HS | HS | HS |
| UACC-812 | HS | NaN | HS |
| ZR-75-30 | LS | LS | LS |

In [54]:
import pandas as pd
import os
import sys
import numpy as np
import glob

# Function for re-filter the out table
# Just filter by total coverage >= 10
def filtering2(path_file, out_dir):
    
    # Create the out_file name
    out_file = os.path.join(out_dir, os.path.basename(path_file).replace('.tsv', '_filtered.tsv'))
    
    count = 0
    with open(path_file, 'r') as fp , open(out_file, 'w+') as fp_out:
        for line in fp:
            if count == 0:
                fp_out.write(f"{line}")
                count += 1
                continue
            count += 1
            tmp = line.strip('\n').split('\t')
            
            line_filt = tmp[:3]
            for col in tmp[3:]:
                if col != 'NA':
                    tmp2 = col.replace('[','').replace(']','').split(',')
                    tmp2 = [int(x) for x in tmp2]
                    if sum(tmp2) < 10:
                        line_filt.append('NA')
                    else:
                        line_filt.append(col)
                else:
                    line_filt.append('NA')
            line_filt2 = line_filt
            line_filt2.append('\n')
            line_filt_out = '\t'.join(line_filt2)
            fp_out.write(f"{line_filt_out}")           
    print('------------------------------------')



# input metadata table and df reading
table = "../data/id_cell_lines.txt"
df = pd.read_csv(table, sep="\t", header=0)
# Recode values
df.replace('High-sensitivity','HS', inplace=True)
df.replace('Low-sensitivity','LS', inplace=True)

# Read the master table
mt = pd.read_csv('../outputs/05_mt-bamreadcount_merge/CCL24_master_table_just-counts_with_bamreadcount.tsv', sep="\t", header=0)
# Get column names
mt_cols = mt.columns
# Create a df with only the columns ['Region','Position','Reference'] and [bam-rc*].
cols2keep = ['Region','Position','Reference']+[x for x in mt_cols if 'bam-rc' in x]
#cols2keep2 = ['Region','Position','Reference']+[x for x in cols2keep if '_HS' in x]+[x for x in mt_cols if '_LS' in x]
mt = mt[cols2keep]

for drug in ['PARPi','Anthracyclines','Alkylatingagents']:
    col_list = ['Region','Position','Reference']
    col_new = ['Region','Position','Reference']
    cell_list = list(df[~df[drug].isna()]['Cell_line'])
    for cell in cell_list:
        # get the group : HS or LS of the cell line on the current drug
        group = df[df['Cell_line'] == cell][drug]
        group = list(group)[0]

        tmp_cols = [x for x in cols2keep if cell in x ]
        new_cols = [x+'_'+group for x in cols2keep if cell in x]
        
        col_list = col_list+tmp_cols
        col_new = col_new+new_cols

    tmp_df = mt[col_list].copy()
    tmp_df.columns = col_new

    # Reorder columns
    cols_order = ['Region','Position','Reference']+[x for x in col_new if '_HS' in x]+[x for x in col_new if '_LS' in x]
    out_df = tmp_df[cols_order]

    # Save tmp file for filtering
    out_df.to_csv(os.path.join('../outputs/05_mt-bamreadcount_merge/', 'tmp.tsv'), sep="\t", header=True, index=False, na_rep='NA')
    
    # Re-filter the table
    filtering2(os.path.join('../outputs/05_mt-bamreadcount_merge/', 'tmp.tsv'), '../outputs/05_mt-bamreadcount_merge/')
    
    
    # Save table
    outname = 'CCLE24_master_table_'+drug+'.tsv'
    print('Saving file {}'.format(outname))
    cmd = "mv ../outputs/05_mt-bamreadcount_merge/tmp_filtered.tsv ../outputs/05_mt-bamreadcount_merge/{}".format(outname)
    os.system(cmd)


/tmp/ipykernel_7225/1178716939.py:51: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  mt = pd.read_csv('../outputs/05_mt-bamreadcount_merge/CCL24_master_table_just-counts_with_bamreadcount.tsv', sep="\t", header=0)


------------------------------------
Saving file CCLE24_master_table_PARPi.tsv
------------------------------------
Saving file CCLE24_master_table_Anthracyclines.tsv
------------------------------------
Saving file CCLE24_master_table_Alkylatingagents.tsv


In [59]:
# Add columns HS_count and LS_count.
# Create list with path to tables
tables = ['../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_PARPi.tsv',
          '../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_Anthracyclines.tsv',
          '../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_Alkylatingagents.tsv']

# For each table
for table in tables:
    # read the table as df
    df = pd.read_csv(table, sep="\t", header=0, na_values='NA')
    # get column names
    cols = df.columns
    # Get col names of HS and LS
    cols_HS = [col for col in cols if "_HS" in col]
    cols_LS = [col for col in cols if "_LS" in col]
    df["HS_NAcount"] = df[cols_HS].isnull().sum(axis=1)
    df["HS_count"] = df[cols_HS].notnull().sum(axis=1)    
    df["LS_NAcount"] = df[cols_LS].isnull().sum(axis=1)
    df["LS_count"] = df[cols_LS].notnull().sum(axis=1)

    # Save the table
    df.to_csv(table.replace('.tsv', '_groupCounts.tsv'), sep="\t", header=True, index=False, na_rep='NA')



/tmp/ipykernel_7225/2652021181.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(table, sep="\t", header=0, na_values='NA')
/tmp/ipykernel_7225/2652021181.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(table, sep="\t", header=0, na_values='NA')
/tmp/ipykernel_7225/2652021181.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(table, sep="\t", header=0, na_values='NA')


# FINAL TABLES

Las tablas finales son :

- `../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_PARPi_groupCounts.tsv`
- `../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_Anthracyclines_groupCounts.tsv`
- `../outputs/05_mt-bamreadcount_merge/CCLE24_master_table_Alkylatingagents_groupCounts.tsv`

A esta tabla se le agregaron 4 columnas al final:

* **HS_NAcount :**  
            Counts of NA in the High Sensitivity Group

* **HS_count :**  
            Counts of VALID in the High Sensitivity Group 

* **LS_NAcount :**  
            Counts of NA in the Low Sensitivity Group

* **LS_count =**  
            Counts of VALID in the Low Sensitivity Group

---

# DO NOT USE: OLD CODE - TEST CODE - ATTEMPS 

## Counting reads with  Samtools
Adicionalmente con ***Samtools*** se puede filtrar un poco los reads antes de contar, tal como se ve en el comando abajo:

`samtools view -@ 10 -q 30 -F 3844 ${file} "${region2}"`

> -q, --min-MQ INT           ...have mapping quality >= INT  
> -F, --excl[ude]-flags FLAG ...have none of the FLAGs present

Donde el FLAG 3844 significa:

> read unmapped (0x4)  
> not primary alignment (0x100)  
> read fails platform/vendor quality checks (0x200)  
> read is PCR or optical duplicate (0x400)  
> supplementary alignment (0x800)  


In [ ]:
%%bash

out_dir="../outputs/04_samtools_counts"
mkdir -p ${out_dir}

# table with editions
table="../outputs/02_master_table/CCLE24_master_table.tsv"

find ../data/bams -name "*.bam" | while read file; 
do 
    echo "Analyzing ${file}"
    outname=$(basename ${file} | sed 's/Aligned.sortedByCoord.out.bam//g')
    awk '{print $1":"$2"-"$3"\t"$4"\t"$5}' $enhancers  | while read region;
    do
        region2=$(echo "$region" | awk '{print $1}')
        id=$(echo "$region" | awk '{print $2}' )
        len=$(echo "$region" | awk '{print $3}')
        count=$(samtools view -@ 10 -q 30 -F 3844 ${file} "${region2}" | wc -l)
        echo -e "${region2}\t${id}\t${len}\t${count}" >> tmp.samtools.count
        
    done        
    mv tmp.samtools.count ${out_dir}/${outname}.counts
done